In [1]:
%matplotlib webagg
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import numpy as np
from matplotlib.axes import Axes
import seaborn as sns
import pandas as pd
plt.style.use("ggplot")

In [2]:
# fig = plt.figure(figsize=(9,7))
fig = plt.figure()
# fig, axs = plt.subplots(2, 2, figsize=(15,15))
# axs: list[Axes] = np.concatenate(axs)
ax1, ax2 = plt.subplot(221), plt.subplot(222)
# [i.set_visible(False) for i in axs[2:]]
ax3 = plt.subplot(212)



# plt.show()
v1, v2 = np.random.rand(2,3)*2
# v1, v2
line1 = ax1.plot([0, v1[0]], [0, v1[1]], "o-")
line2 = ax1.plot([0, v2[0]], [0, v2[1]], "o-")
ax1.set_xlim(-2,2)
ax1.set_ylim(-2,2)
ax1.grid(True)
ax1.set_title("2D projection of data vectors")

line3 = ax3.plot(range(1, len(v1)+1),v1, "o-")
line4 = ax3.plot(range(1, len(v2)+1),v2, "o-")
ax3.set_xlim(0.5,len(v1)+0.5)
ax3.set_ylim(-2,2)
ax3.grid(True)
ax3.set_title("Line plot of data")
# ax1.set_aspect(1)

def similarities(v1, v2):
    cos_sim = sum(v1*v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
    cov = np.cov([v1, v2])[0,1]
    cor = np.corrcoef([v1,v2])[0,1]
    euclid_dist = np.linalg.norm(v1-v2)
    manhatan_dist = sum([abs(i) for i in v1-v2])
    return {
        "cos\nsimilarity": [cos_sim],
        "covariance": [cov],
        "\ncorrelation": [cor],
        "euclidian\ndistance": [euclid_dist],
        "manhattan\ndistance": [manhatan_dist]
    }
def update(i):
    # print("Updating")
    # ax1, ax2, ax3, ax4 = axs[:]
    # ax1.clear()
    # ax3.clear()
    line1[0].set_data([0, v1[0]], [0, v1[1]])
    line2[0].set_data([0, v2[0]], [0, v2[1]])
    df = pd.DataFrame(similarities(v1, v2))
    ax2.clear()
    ax2.set_title("Distances and similarities")
    sns.barplot(df, ax=ax2)
    ax2.grid(True)
    line3[0].set_data(range(1, len(v1)+1),v1)
    line4[0].set_data(range(1, len(v2)+1),v2)

    
dragging = [None]
def mouse_move(e):
    if dragging[0] is None:
        return
    if e.inaxes==ax1:
        v = dragging[0]
        v[0] = e.xdata
        v[1] = e.ydata
    if e.inaxes==ax3:
        i,v = dragging[0]
        v[i] = e.ydata
def mouse_press(e):
    loc = np.array([e.xdata,e.ydata])
    if e.inaxes==ax1:
        near_v = min([v1,v2], key=lambda i: np.linalg.norm(i[:2]-loc))
        if np.linalg.norm(near_v[:2]-loc) < 0.1:
            dragging[0] = near_v
    if e.inaxes==ax3:
        i,v = min(sum([list(enumerate([v]*len(v))) for v in [v1,v2]],[]), key=lambda a:np.linalg.norm([a[0]+1,a[1][a[0]]]-loc))
        if np.linalg.norm([i+1,v[i]]-loc) < 0.1:
            dragging[0]=(i,v)
def mouse_unpress(e):
    dragging[0] = None
fig.canvas.mpl_connect("button_press_event", mouse_press)
fig.canvas.mpl_connect("button_release_event", mouse_unpress)
fig.canvas.mpl_connect("motion_notify_event", mouse_move)
fig.tight_layout()
None

In [3]:
anime = anim.FuncAnimation(fig, update, 100)
plt.show()